In [1]:
!pip install -q -U kaggle_environments

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [10]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

1) Агент, который играет только камнями

In [11]:
%%writefile rock_agent.py

def rock_agent(observation, configuration):
    return 0


Overwriting rock_agent.py


2) Агент, который играет только бумагой

In [12]:
%%writefile paper_agent.py

def paper_agent(observation, configuration):
    return 1


Overwriting paper_agent.py


3) Агент, который играет только ножницами

In [13]:
%%writefile scissors_agent.py

def scissors_agent(observation, configuration):
    return 2


Overwriting scissors_agent.py


4) Агент, который повторяет последний ход противника

In [14]:
%%writefile copy_opponent.py
import random

def copy_opponent(observation, configuration):
    if observation.step > 0:
        return observation.lastOpponentAction
    else:
        return random.randint(0, 2)


Overwriting copy_opponent.py


5) Агент, который совершает выигрышный ход на прошлый ход противника

In [15]:
%%writefile do_opposite.py
import random

def find_opposite(choice):
    return (choice + 1) % 3

def do_opposite(observation, configuration):
    if observation.step > 0:
        return find_opposite(observation.lastOpponentAction)
    else:
        return random.randint(0, 2)


Overwriting do_opposite.py


6) Агент, который выбирает камень, ножницы, бумагу случайно

In [16]:
%%writefile do_random.py
import random

def do_random(observation, configuration):
    return random.randint(0, 2)


Overwriting do_random.py


7) Агент, который выбирает камень, ножницы, бумагу по кругу

In [17]:
%%writefile cycle_agent.py
from itertools import cycle

c = cycle([0, 1, 2])

def cycle_agent(observation, configuration):
    global c
    
    if observation.step == 0:
        c = cycle([0, 1, 2])
    
    return next(c)


Overwriting cycle_agent.py


8) Агент, который выбирает случайное действие, основываясь на предыдущих ходах противника, и совершает выигрышное действие на него

In [18]:
%%writefile choice_random_from_actions_opponent.py
import random

actions = []

def find_opposite(choice):
    return (choice + 1) % 3

def choice_random_from_actions_opponent(observation, configuration):
    global actions
    
    if observation.step > 0:
        actions.append(observation.lastOpponentAction)
        return find_opposite(random.choice(actions))
    else:
        actions = []
        return random.randint(0, 2)


Overwriting choice_random_from_actions_opponent.py


9) Агент, который совершает только выигрышное действие на самый первый ход противника

In [19]:
%%writefile first_action_agent.py
import random

first_action = None

def find_opposite(choice):
    return (choice + 1) % 3

def first_action_agent(observation, configuration):
    global first_action
    
    if observation.step == 1:
        first_action = find_opposite(observation.lastOpponentAction)
    if observation.step > 0:
        return first_action
    else:
        first_action = None
        return random.randint(0, 2)


Overwriting first_action_agent.py


10) Агент, который выбирает случайное действие между камнем и бумагой

In [20]:
%%writefile choose_random_from_two.py
import random

def choose_random_from_two(observation, configuration):
    return random.randint(0, 1)

Overwriting choose_random_from_two.py


11) Агент, который играет "елочкой" с циклами, то есть ходы его следующие (подряд):

0

1 1

2 2 2

0 0 0 0

1 1 1 1 1

...

In [53]:
%%writefile christmas_tree_agent.py
from itertools import count, cycle

def christmas_tree():
    it_count = count(1)
    it_cycle = cycle((0, 1, 2))
    act = next(it_cycle)

    for i in it_count:
        for j in range(i):
            yield act
        act = next(it_cycle)

it_christmas_tree = christmas_tree()

def christmas_tree_agent(observation, configuration):
    global it_christmas_tree

    if observation.step == 0:
        it_christmas_tree = christmas_tree()

    return next(it_christmas_tree)


Overwriting christmas_tree_agent.py


12) Агент, который выбирает самое частое действие противника, и совершает выигрышное на него

In [22]:
%%writefile agent_the_most_common.py
import random

actions = {}

def find_opposite(choice):
    return (choice + 1) % 3

def choose_actions(actions):
    m = max(actions.values())
    for k, value in actions.items():
        if value == m:
            return find_opposite(k)

def agent_the_most_common(observation, configuration):
    global actions
    
    if observation.step == 0:
        actions = {}
        return random.randint(0, 2)
    else:
        last_action = observation.lastOpponentAction
        if last_action not in actions:
            actions[last_action] = 0
        actions[last_action] += 1
        return choose_actions(actions)
        

Writing agent_the_most_common.py


13) Агент, который выбирает самое редкое действие действие противника

In [65]:
%%writefile agent_the_least_common.py
import random

actions = {}

def choose_actions(actions):
    m = min(actions.values())
    for k, value in actions.items():
        if value == m:
            return k

def agent_the_least_common(observation, configuration):
    global actions
    
    if observation.step == 0:
        actions = {}
        return random.randint(0, 2)
    else:
        last_action = observation.lastOpponentAction
        if last_action not in actions:
            actions[last_action] = 0
        actions[last_action] += 1
        return choose_actions(actions)
        

Writing agent_the_least_common.py


In [69]:
evaluate(
    "rps",
    ["rock_agent.py", "paper_agent.py"],
    configuration={"episodeSteps": 1000}
)

[[-999.0, 999.0]]

In [75]:
evaluate(
    "rps",
    ["scissors_agent.py", "copy_opponent.py"],
    configuration={"episodeSteps": 1000}
)

[[0, 0]]

In [71]:
evaluate(
    "rps",
    ["do_opposite.py", "do_random.py"],
    configuration={"episodeSteps": 1000}
)

[[-52.0, 52.0]]

In [74]:
evaluate(
    "rps",
    ["cycle_agent.py", "choice_random_from_actions_opponent.py"],
    configuration={"episodeSteps": 1000}
)

[[-21.0, 21.0]]

In [76]:
evaluate(
    "rps",
    ["first_action_agent.py", "choose_random_from_two.py"],
    configuration={"episodeSteps": 1000}
)

[[508.0, -508.0]]

In [79]:
evaluate(
    "rps",
    ["christmas_tree_agent.py", "agent_the_most_common.py"],
    configuration={"episodeSteps": 1000}
)

[[-315.0, 315.0]]

In [81]:
evaluate(
    "rps",
    ["agent_the_least_common.py", "paper_agent.py"],
    configuration={"episodeSteps": 1000}
)

[[0, 0]]

In [82]:
evaluate(
    "rps",
    ["scissors_agent.py", "do_random.py"],
    configuration={"episodeSteps": 1000}
)

[[-33.0, 33.0]]

In [83]:
evaluate(
    "rps",
    ["copy_opponent.py", "do_random.py"],
    configuration={"episodeSteps": 1000}
)

[[0, 0]]

In [84]:
evaluate(
    "rps",
    ["first_action_agent.py", "choice_random_from_actions_opponent.py"],
    configuration={"episodeSteps": 1000}
)

[[-981.0, 981.0]]

In [85]:
evaluate(
    "rps",
    ["agent_the_least_common.py", "agent_the_most_common.py"],
    configuration={"episodeSteps": 1000}
)

[[-998.0, 998.0]]

In [86]:
evaluate(
    "rps",
    ["scissors_agent.py", "paper_agent.py"],
    configuration={"episodeSteps": 1000}
)

[[999.0, -999.0]]

In [87]:
evaluate(
    "rps",
    ["copy_opponent.py", "choice_random_from_actions_opponent.py"],
    configuration={"episodeSteps": 1000}
)

[[0, 0]]

In [88]:
evaluate(
    "rps",
    ["do_random.py", "choice_random_from_actions_opponent.py"],
    configuration={"episodeSteps": 1000}
)

[[0, 0]]

In [89]:
evaluate(
    "rps",
    ["scissors_agent.py", "agent_the_most_common.py"],
    configuration={"episodeSteps": 1000}
)

[[-997.0, 997.0]]

In [90]:
evaluate(
    "rps",
    ["copy_opponent.py", "agent_the_most_common.py"],
    configuration={"episodeSteps": 1000}
)

[[-201.0, 201.0]]

In [91]:
evaluate(
    "rps",
    ["choice_random_from_actions_opponent.py", "agent_the_most_common.py"],
    configuration={"episodeSteps": 1000}
)

[[29.0, -29.0]]

In [92]:
evaluate(
    "rps",
    ["do_random.py", "agent_the_most_common.py"],
    configuration={"episodeSteps": 1000}
)

[[27.0, -27.0]]

In [93]:
evaluate(
    "rps",
    ["choice_random_from_actions_opponent.py", "do_random.py"],
    configuration={"episodeSteps": 1000}
)

[[-50.0, 50.0]]

# Победил агент, который выбирал случайное действие